In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [2]:
import datetime
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

In [3]:
# download an hour of THEMIS ASI data
dataset_name = "THEMIS_ASI_RAW"
start_dt = datetime.datetime(2021, 11, 4, 9, 0)
end_dt = datetime.datetime(2021, 11, 4, 9, 59)
site_uid = "atha"
r = aurorax.data.ucalgary.download(dataset_name, start_dt, end_dt, site_uid=site_uid)

In [4]:
# read in the hour of data
data = aurorax.data.ucalgary.read(r.dataset, r.filenames, n_parallel=5)

In [5]:
# set up working with the tools by just making a shorter name for our future calls
at = aurorax.tools

In [6]:
# scale all the images
#
# NOTE: you can scale all images or just one image
images_scaled = at.scale_intensity(data.data, min=1000, max=10000)

In [7]:
# we're going to process all frames from the hour. We'll use the display() function
# and have it return the figure, then we'll add some timestamp and other text
# to our liking.
#
# we'll utilize multiprocessing and tqdm's progress bar functionality to do this
# using several worker processes

import os
import platform
import matplotlib.pyplot as plt
from tqdm.contrib.concurrent import process_map as tqdm_process_map
from tqdm.auto import tqdm


def process_frame(i):
    fig, ax = at.display(images_scaled[:, :, i], cmap="gray", returnfig=True)
    ax.text(5, 240, "THEMIS ASI", color="white", size=14)
    ax.text(5, 225, "ATHA", color="white", size=14)
    ax.text(145, 8, data.timestamp[i].strftime("%Y-%m-%d %H:%M:%S UTC"), color="white", size=11)
    filename = "movie_frames/%s_atha_themis.png" % (data.timestamp[i].strftime("%Y%m%d_%H%M%S"))
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    plt.savefig(filename, bbox_inches="tight")
    plt.close()
    return filename


if (platform.system() == "Windows"):
    # pre-process frames serially
    #
    # NOTE: multiprocessing on Windows from within a notebook is not too easy, so we'll
    # just do this serially.
    frame_filename_list = []
    for i in tqdm(range(0, images_scaled.shape[-1]), total=images_scaled.shape[-1], desc="Generating frame files: ", unit="frames"):
        frame_filename_list.append(process_frame(i))
else:
    frame_filename_list = tqdm_process_map(
        process_frame,
        range(0, images_scaled.shape[-1]),
        max_workers=5,
        chunksize=1,
        desc="Generating frame files: ",
        unit="frames",
    )

Generating frame files:   0%|          | 0/1200 [00:00<?, ?frames/s]

In [8]:
# now that we have our frames, we'll generate a movie using them
at.movie(frame_filename_list, "test.mp4", n_parallel=5)

Reading files:   0%|          | 0/1200 [00:00<?, ?files/s]

Encoding frames:   0%|          | 0/1200 [00:00<?, ?frames/s]